In [96]:
# imports
import pandas as pd

from copy import copy
from openpyxl import load_workbook
from openpyxl.cell import Cell
from openpyxl.formula.translate import Translator
from openpyxl.workbook import workbook
from openpyxl.worksheet import worksheet
from pandas import DataFrame
from sqlalchemy import create_engine

# configuration
sql_dialect = 'mssql'
sql_driver = 'pyodbc'
sql_conn = '@./Goal-Based-Database?'
driver = 'driver=ODBC+Driver+17+for+SQL+Server'

# inputs
template_name = 'Template.xlsx'
output_name = 'Report.xlsx'

sql1 = 'SELECT Cost, 0.0 FROM CashFlow'
sh_name1 = 'CashFlow'
start_row1 = 2

sql2 = 'SELECT TOP 10 Cost/2.0, \'Id\' + CONVERT(VARCHAR, Id) FROM CashFlow'
sh_name2 = 'CashFlowV2'
start_row2 = 3

# classes
class Config():
    def __init__(self, sql_dialect, sql_driver, sql_conn, driver):
        self.sql_dialect = sql_dialect
        self.sql_driver = sql_driver
        self.sql_conn = sql_conn
        self.driver = driver
        
    def get_engine_conn(self) -> str:
        return self.sql_dialect + '+' + self.sql_driver + '://' + self.sql_conn + self.driver
        
class ReportContext():    
    def __init__(self, template_name, output_name):
        self.template_name = template_name
        self.output_name = output_name
        self.sheet_contexts = []
        
    def add_sheet(self, sql, sh_name, start_row):
        sheet_context = SheetContext(sql, sh_name, start_row)
        self.sheet_contexts.append(sheet_context)
        
class SheetContext():
    def __init__(self, sql, sh_name, start_row):
        self.sql = sql
        self.sh_name = sh_name
        self.start_row = start_row        

# functions
def export_excel(report_context:ReportContext) -> workbook:
    def get_populated_worksheet(ws: worksheet, df: DataFrame, start_row: int) -> worksheet:
        cols = df.shape[1]
        rows = df.shape[0]
        for col in range(cols):
            reference = ws.cell(row=start_row, column=col + 1)
            for row in range(rows):
                cell = ws.cell(row=row+start_row+1, column=col+1)
                cell.value = get_cell_value(ws, df, cell, reference)
                cell = get_formatted_cell(cell,reference)
        ws.delete_rows(start_row)
        return ws
    
    def get_cell_value(ws: worksheet, df: DataFrame, cell: Cell, reference: Cell):
        if reference.data_type == 'f':
            coordinate = ws.cell(row=cell.row - 1, column=cell.column).coordinate
            return Translator(reference.value, reference.coordinate).translate_formula(coordinate)
        else:
            df_row = cell.row - reference.row - 1
            return df.values[df_row,cell.column - 1]

    def get_formatted_cell(cell: Cell, reference: Cell) -> Cell:
        if reference.has_style:
            cell.font = copy(reference.font)
            cell.border = copy(reference.border)
            cell.fill = copy(reference.fill)
            cell.number_format = copy(reference.number_format)
            cell.protection = copy(reference.protection)
            cell.alignment = copy(reference.alignment)
        return cell
    
    wb = load_workbook(filename = report_context.template_name)
    for sheet_context in report_context.sheet_contexts:
        df = pd.read_sql(sheet_context.sql, engine)
        ws = wb[sheet_context.sh_name]
        ws = get_populated_worksheet(ws, df, sheet_context.start_row)
    wb.save(report_context.output_name)
    return wb
           
# execute
config = Config(sql_dialect, sql_driver, sql_conn, driver)
engine = create_engine(config.get_engine_conn())

report_context = ReportContext(template_name, output_name)
report_context.add_sheet(sql1, sh_name1, start_row1)
report_context.add_sheet(sql2, sh_name2, start_row2)

excel_export = export_excel(report_context)